## 希土類Co合金の磁気転移温度



***説明変数***

周期律表

- Z : atomic number

構造、構造に関する説明変数はAtomWorkから結晶構造を取得した後に加工している。


- C_R : 希土類元素の数密度
- C_T : Coの数密度
- vol_per_atom: 元素あたりの体積

希土類原子説明変数

- f4, d5: 希土類元素の電子配置
- L4f, S4f, J4f,(g-1)J4f, (2-g)J4f ,　元素の角運動量期待値とその射影量

***目的変数***

- Tc: Curie temperature （K)


In [ ]:
INPUT_DATA_DIR = "../data"
OUTPUT_IMG_DIR = "image_executed"
IMG_FORMAT = "png"

In [ ]:
import pandas as pd
import os

pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",100)
import matplotlib.pyplot as plt


In [ ]:
!head -3 ../data/TC_ReCo_detail_descriptor.csv

In [ ]:
df = pd.read_csv(os.path.join(INPUT_DATA_DIR,"TC_ReCo_detail_descriptor.csv"))
descriptor_names = ['C_R', 'C_T', 'vol_per_atom', 'Z', 'f4', 'd5', 'L4f', 'S4f', 'J4f',
       '(g-1)J4f', '(2-g)J4f']
target_name = 'Tc'
mata_labels = ['name', 'polytyp','ref', 'author', 'link', 'comment', 'polytyp2']

In [ ]:
df.head()

In [ ]:
def show_hist(df, target_name, output_img_dir=OUTPUT_IMG_DIR,img_format=IMG_FORMAT):
    fig, ax = plt.subplots(figsize=(4,4))
    df.hist(target_name, ax=ax)
    ax.set_xlabel(target_name)
    ax.set_ylabel("DOS")
    fig.tight_layout()
    if True:
        os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)
        fig.savefig(os.path.join(output_img_dir,f"TC_RECo_Tc_historgram.{img_format}"))
    fig.show()
    
show_hist(df, target_name)

In [ ]:
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist,squareform
from scipy.cluster.hierarchy import dendrogram, linkage
import copy
from scipy.stats import pearsonr
import numpy as np

def make_linkage(df, descriptor_names, target_name, corr="minus_abs_pearson", method="weighted"):
    labels = copy.deepcopy(descriptor_names)
    labels.append(target_name)
    Xraw = df.loc[:,labels].values
    scaler = StandardScaler()
    X = scaler.fit_transform(Xraw)
    df_tmp = pd.DataFrame(X)
    if corr=="minus_abs_pearson":
        corr = 1- np.abs(df_tmp.corr())
    else:
        raise ValueError("unknown corr={}".format(corr))
    pairdistance = squareform(corr)
    Z = linkage(pairdistance, method=method)
    return Z, labels

def show_dendrogram(Z,labels, corr, output_img_dir=OUTPUT_IMG_DIR, img_format=IMG_FORMAT):
    fig, ax = plt.subplots()
    dendrogram(Z,labels=labels,orientation="left", ax=ax)
    ax.set_xlabel(corr)
    fig.tight_layout()
    if True:
        os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)
        fig.savefig(os.path.join(output_img_dir,f"TC_RECo_{corr}_dendrogram.{img_format}"))
    fig.show()
    
corr="minus_abs_pearson"
Z, labels = make_linkage(df, descriptor_names, target_name, corr)
show_dendrogram(Z,labels, corr)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def plot_z_Tc(df, output_img_dir=OUTPUT_IMG_DIR,img_format=IMG_FORMAT):
    color = ["black","gray","silver","red","darkkhaki","darksalmon",
             "gold","green","blue","deepskyblue","m","paleviotedred"]
    plt.figure()
    labels = np.unique(df["polytyp2"].values)
    for i,label in enumerate(labels):
        dfq = df.query("polytyp2=='{}'".format(label)).sort_values(by="Z")
        x = dfq.loc[:,'Z'].values
        y = dfq.loc[:,'Tc'].values
        plt.plot(x,y,"o-",c=color[i],label=label)
    plt.xlabel("Z")
    plt.ylabel("Tc")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0 )
    plt.tight_layout()
    if True:
        os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)
        plt.savefig(os.path.join(output_img_dir,f"TC_RECo_vs_Z.{img_format}"))
    plt.show()
plot_z_Tc(df)

In [ ]:
import os
import seaborn as sns
from copy import deepcopy

imgfile = os.path.join(OUTPUT_IMG_DIR,f"ReCo_pairplot.{IMG_FORMAT}")
# すでにimgfileが有る場合は再度実行しない。
if not os.path.isfile(imgfile):
    alllabels = deepcopy(descriptor_names)
    alllabels.append(target_name)
    img = sns.pairplot(df[alllabels])
    os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)
    img.savefig(imgfile)

# imgfileを表示する。
import IPython
IPython.display.Image(imgfile)

**参考文献**

1. Hieu Chi Dam, Viet Cuong Nguyen, Tien Lam Pham, Anh Tuan Nguyen, Kiyoyuki Terakura, Takashi Miyake, and Hiori Kino, J. Phys. Soc. Jpn. 87, 113801 (2018). (open select) https://journals.jps.jp/doi/abs/10.7566/JPSJ.87.113801
2. Tcの取得先論文に関してはファイル内に記載しています。
3. 量子数関連説明変数 Jens Jensen and Allan R. Mackintosh, "Rare Earth Magnetism, Structures and Excitations", Clarendon Press, Oxford(1991)
4. 構造の元データ取得元。
http://crystdb.nims.go.jp/　（June, 28, 2017), 
Yibin Xu, Masayoshi Yamazaki, and Pierre Villars: Inorganic Materials Database for Exploring the Nature of Material: Jpn. J. Appl. Phys. 50 (2011) 11RH02.